In [ ]:
import pandas as pd
import os
from supabase import create_client, Client
from dotenv import load_dotenv
load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [7]:
# Folder path
csv_folder = "hugo_data_samples"

# Loop through all CSV files
for filename in os.listdir(csv_folder):
    if filename.endswith(".csv"):
        csv_path = os.path.join(csv_folder, filename)
        csv_name = os.path.splitext(filename)[0]
        
        # Read CSV
        df = pd.read_csv(csv_path)

        # Build CREATE TABLE SQL
        columns_sql = []
        for col_name, dtype in zip(df.columns, df.dtypes):
            if pd.api.types.is_integer_dtype(dtype):
                sql_type = "INTEGER"
            elif pd.api.types.is_float_dtype(dtype):
                sql_type = "FLOAT"
            elif pd.api.types.is_bool_dtype(dtype):
                sql_type = "BOOLEAN"
            else:
                sql_type = "TEXT"
            columns_sql.append(f'"{col_name}" {sql_type}')

        columns_sql_string = ", ".join(columns_sql)
        create_table_sql = f"""
            CREATE TABLE IF NOT EXISTS "{csv_name}" (
                id SERIAL PRIMARY KEY,
                {columns_sql_string}
            );
        """

        # Execute SQL
        response = supabase.rpc('execute_sql', {'sql': create_table_sql}).execute()